<a href="https://colab.research.google.com/github/emrahe82/emrahVHS/blob/main/elasticsearch_demo_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [1]:
! pip install elasticsearch==7
! pip install gcsfs
! pip install json5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 8.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import Libraries

In [2]:
from datetime import datetime
from elasticsearch import Elasticsearch

### ElasticSearch credentials

In [3]:
ES_URL = "https://my-deployment-460646.es.us-central1.gcp.cloud.es.io:9243" # "https://aaa-a725c6.es.us-central1.gcp.cloud.es.io:9243"
ES_USER = 'elastic'
ES_PASSWORD = "Y7y8I8UQp26EIIMmXawqB3zC"
PROD_ES_URL = "https://my-deployment-460646.es.us-central1.gcp.cloud.es.io:9243"
PROD_DEVELOPERS_INDEX_NAME = "search-emrah-text-search"

### Configure the ES Client

In [4]:
from elasticsearch import Elasticsearch, helpers

es = Elasticsearch(hosts = [ES_URL], http_auth=(ES_USER, ES_PASSWORD), timeout = 60)
print(es)

<Elasticsearch([{'host': 'my-deployment-460646.es.us-central1.gcp.cloud.es.io', 'port': 9243, 'use_ssl': True}])>


In [7]:
from elasticsearch import Elasticsearch

es = Elasticsearch(hosts = [ES_URL], http_auth=(ES_USER, ES_PASSWORD), timeout = 60)

# Ping the Elasticsearch server
if es.ping():
    print("Connected to the Elasticsearch server")
else:
    print("Could not connect to the Elasticsearch server")


Connected to the Elasticsearch server


In [ ]:
es.indices.create

### Trial Codebase

In [8]:
request_body = {
	    "settings" : {
	        "number_of_shards": 5,
	        "number_of_replicas": 1
	    },
"mappings" : {
        "properties": {
            "title": {"type": "text", "analyzer": "english"},
            "ethnicity": {"type": "text", "analyzer": "standard"},
            "director": {"type": "text", "analyzer": "standard"},
            "cast": {"type": "text", "analyzer": "standard"},
            "genre": {"type": "text", "analyzer": "standard"},
            "plot": {"type": "text", "analyzer": "english"},
            "year": {"type": "integer"},
            "wiki_page": {"type": "keyword"}
    }
}
  }
if not es.indices.exists(index="vhs_tapes"):
  es.indices.create(index="vhs_tapes", body = request_body)

In [9]:
import pandas as pd

In [10]:
df = pd.DataFrame({
    "title": ["a test", "man made", "cyclone", "cyclone war"],
    "ethnicity": ["same"] * 4,
    "director": ["test", "fahad", "emrah", "nabil"],
    "cast": ["test", "fahad", "emrah", "nabil"],
    "genre": ["test", "fahad", "emrah", "nabil"],
    "plot": ["test", "fahad", "emrah", "nabil"],
    "year": [1998, 2002, 2005, 2020],
    "wiki_page": ["test", "fahad", "emrah", "nabil"],

})

In [11]:
df.head()

,title,ethnicity,director,cast,genre,plot,year,wiki_page
0,a test,same,test,test,test,test,1998,test
1,man made,same,fahad,fahad,fahad,fahad,2002,fahad
2,cyclone,same,emrah,emrah,emrah,emrah,2005,emrah
3,cyclone war,same,nabil,nabil,nabil,nabil,2020,nabil


In [12]:
for i, row in df.iterrows():
    doc = {
        "title": row["title"],
        "ethnicity": row["ethnicity"],
        "director": row["director"],
        "cast": row["cast"],
        "genre": row["genre"],
        "plot": row["plot"],
        "year": row["year"],
        "wiki_page": row["wiki_page"]
    }

    es.index(index="vhs_tapes", id=i, body=doc)

### Final CodeBase

In [14]:
def create_index(index_name):
    # Create an index in Elasticsearch with a custom analyzer
    analyzer = {
        'analyzer': {
            'my_analyzer': {
                'tokenizer': 'standard',
                'filter': ['lowercase']
            }
        }
    }
    es.indices.create(index=index_name, ignore=400, body=analyzer)

def index_document(index_name, document_id, text):
    # Index a document in Elasticsearch
    body = {
        'text': text
    }
    es.index(index=index_name, id=document_id, body=body)

def search_documents(index_name, query):
    # Search for documents in Elasticsearch
    body = {
        'query': {
            'match': {
                'text': {
                    'query': query,
                    'analyzer': 'standard'
                }
            }
        }
    }
    response = es.search(index=index_name, body=body)

    # Extract relevant information from the response
    hits = response['hits']['hits']
    search_results = []
    for hit in hits:
        document_id = hit['_id']
        score = hit['_score']
        search_results.append({'document_id': document_id, 'score': score})

    return search_results

# Example usage

# Create an index
index_name = 'my_documents'
create_index(index_name)

# Index some documents
index_document(index_name, 1, 'The quick brown fox')
index_document(index_name, 2, 'Jumped over the lazy dog')
index_document(index_name, 3, 'Hello world')

# Search for documents
query = 'brown'
results = search_documents(index_name, query)

# Display search results
print(f"Search results for query '{query}':")
for result in results:
    print(f"Document ID: {result['document_id']}, Score: {result['score']}")

Search results for query 'brown':
Document ID: 1, Score: 0.94566
